In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import numpy as np

# Lue CSV-tiedosto
file_path = r"D:\Koulu\AIDA_S24\AIDA_project\aida-projekti-local\archive\used_cars_data.csv"
df = pd.read_csv(file_path, nrows=5000, usecols=[
    'back_legroom', 'body_type', 'city', 'city_fuel_economy', 'daysonmarket', 
    'engine_displacement', 'engine_type', 'fleet', 'front_legroom', 
    'fuel_tank_volume', 'fuel_type', 'has_accidents', 'height',
    'highway_fuel_economy', 'horsepower', 'isCab', 'is_new', 'latitude', 
    'length', 'longitude', 'major_options', 'make_name', 'maximum_seating', 
    'mileage', 'model_name', 'price', 'seller_rating', 'transmission_display', 
    'wheel_system_display', 'width', 'year'
])

print("days on market keskiarvo", df['daysonmarket'].mean())

# Muunna daysonmarket nopeaan 1 ja hitaaseen 0 myyntiaikaan, nopea on alle 60pv kaupassa
df['quick_sale'] = df['daysonmarket'].apply(lambda x: 1 if x < 80 else 0)

df = df.drop(columns=['daysonmarket'])

# Poista sarakkeet joissa on pelkästään puuttuvia arvoja
df = df.dropna(axis=1, how='all')

X = df.drop(columns=['quick_sale'])
y = df['quick_sale']

ykkoset = y.values.sum()
print(ykkoset)

# Määrittele esikäsittelyvaiheet
numerical_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object', 'bool']).columns

# Määritellään imputerit ja muuntajat numeerisille ja kategorisille arvoille
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())  # Skaalaa numeeriset arvot
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Yhdistetään muuntajat
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Luo pipeline, jossa on esikäsittely ja RandomForestClassifier
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', RandomForestClassifier(random_state=42))])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Laske ja tulosta tarkkuus
print("Test accuracy: %.2f" % model.score(X_test, y_test))

# Tulosta sekaannusmatriisi ja luokitteluraportti
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Hae ominaisuuksien tärkeysarvot
importances = model.named_steps['classifier'].feature_importances_

# Sarakenimet, mukaan lukien one-hot-koodatut kategoriset muuttujat
feature_names = model.named_steps['preprocessor'].transformers_[0][2].tolist() + \
                list(model.named_steps['preprocessor'].transformers_[1][1]['onehot'].get_feature_names_out(categorical_features))

for i in range(len(importances)):
    if importances[i] > 0.01:
        print("Feature: %20s - Relative importance: %4.1f%%" % (feature_names[i], importances[i] * 100))

days on market keskiarvo 79.0594
3726
Test accuracy: 0.86
[[ 214  148]
 [  66 1072]]
              precision    recall  f1-score   support

           0       0.76      0.59      0.67       362
           1       0.88      0.94      0.91      1138

    accuracy                           0.86      1500
   macro avg       0.82      0.77      0.79      1500
weighted avg       0.85      0.86      0.85      1500

Feature:    city_fuel_economy - Relative importance:  1.5%
Feature:  engine_displacement - Relative importance:  1.4%
Feature: highway_fuel_economy - Relative importance:  1.7%
Feature:           horsepower - Relative importance:  1.9%
Feature:             latitude - Relative importance:  1.8%
Feature:            longitude - Relative importance:  1.8%
Feature:              mileage - Relative importance:  7.7%
Feature:                price - Relative importance: 10.1%
Feature:        seller_rating - Relative importance:  1.6%
Feature:                 year - Relative importance:  4.5